### MAP

In [5]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
# fp = 'statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'
fp = 'statistical-gis-boundaries-london/ESRI/London_Ward_CityMerged.shp'
# fp = 'statistical-gis-boundaries-london/ESRI/London_Ward.shp'
map_df = gpd.read_file(fp)

ward=map_df[['NAME', 'GSS_CODE', 'LB_GSS_CD', 'BOROUGH']]
print(ward.shape)

ward['NAME_St']=ward['NAME'].str.replace('.', '')
ward.head()

(625, 4)


/Users/tanyanabila/Applications/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley


In [10]:
df=pd.read_excel('CT0226 - Country of birth (detailed).xls', sheet_name='df', header=None)

confusing_wards=[]
for i in df[0]:
    i=i.lstrip()
    if(i.startswith('E09')):
        borough_code=i.split(' ')[0]
        borough_name=i.split(' ', 1)[1]
    if(i.startswith('E36')):
        ward_code=i.split(' ')[0]
        ward_name=i.split(' ', 1)[1]
        confusing_wards.append([borough_code, borough_name, ward_code, ward_name])
        
df_confusing_wards=pd.DataFrame(confusing_wards)
df_confusing_wards.columns=['borough_code', 'borough_name', 'weird_ward_code', 'ward_name']
df_confusing_wards.head()
# df_confusing_wards.to_csv('df_confusing_wards.csv')

tmp=pd.merge(ward, df_confusing_wards, how='left', left_on=['NAME_St', 'LB_GSS_CD'], right_on=['ward_name', 'borough_code'])
tmp.head()

ward_all=tmp[['NAME',
              'GSS_CODE', 
              'LB_GSS_CD', 
              'BOROUGH',
              'NAME_St',
              'weird_ward_code']]

### Gender, Age & Ethnicity

In [11]:
df_age=pd.read_excel('Age_percentage.xlsx', sheet_name = "df")
print(df_age.shape)
df_age.head()

(624, 11)


,Code,Name,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,16_24,25_49,50_64,65_74,75_over
0,E36007055,Abbey,0.857579,0.092367,0.050055,0.244408,0.141796,0.471375,0.092367,0.030189,0.019865
1,E36007056,Alibon,0.759942,0.136254,0.103804,0.264035,0.121810,0.374097,0.136254,0.052480,0.051324
2,E36007057,Becontree,0.775227,0.130013,0.094760,0.254396,0.128974,0.391858,0.130013,0.048419,0.046340
3,E36007058,Chadwell Heath,0.705319,0.138609,0.156072,0.245085,0.115857,0.344377,0.138609,0.067159,0.088913
4,E36007059,Eastbrook,0.683324,0.169808,0.146868,0.212450,0.136589,0.334285,0.169808,0.069008,0.077860


In [12]:
merged1 = pd.merge(ward_all, df_age, how='left', left_on='weird_ward_code', right_on='Code')
merged1 = merged1.drop(['Code', 'Name'], axis=1)
merged1.head()

,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St,weird_ward_code,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,16_24,25_49,50_64,65_74,75_over
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South,E36007434,0.690188,0.174941,0.134871,0.206802,0.107213,0.376173,0.174941,0.071443,0.063428
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise,E36007443,0.703854,0.167192,0.128954,0.204617,0.113597,0.385640,0.167192,0.067426,0.061528
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands,E36007430,0.699375,0.151425,0.149200,0.169228,0.117728,0.412419,0.151425,0.071421,0.077779
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra,E36007429,0.657906,0.193705,0.148388,0.208631,0.111400,0.337876,0.193705,0.078845,0.069544
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley,E36007431,0.701454,0.169453,0.129093,0.206648,0.112276,0.382530,0.169453,0.062024,0.067069


In [13]:
df_ethnicity=pd.read_excel('Ethnicity_percentage.xlsx', sheet_name = "df")
df_ethnicity.head()

,Code,Name,Risky,No_Risk,White,White African,Black African,Pakistani,Bangladeshi,Indian,Caribbean,Chinese,Latin,Polish,Other,Other_Asian,Other_Black
0,E36007055,Abbey,0.629047,0.370953,0.254575,0.019240,0.162834,0.174018,0.067105,0.133114,0.039887,0.012592,0.003911,0.014234,0.051228,0.052088,0.015173
1,E36007056,Alibon,0.257679,0.742321,0.671353,0.011940,0.153009,0.010977,0.023688,0.011459,0.032836,0.006163,0.001830,0.007318,0.032932,0.025710,0.010785
2,E36007057,Becontree,0.351351,0.648649,0.565229,0.014466,0.133836,0.054401,0.035343,0.043486,0.051455,0.007450,0.001646,0.008663,0.037855,0.032831,0.013340
3,E36007058,Chadwell Heath,0.310379,0.689621,0.609481,0.015070,0.096407,0.036926,0.030639,0.053094,0.065369,0.006587,0.003293,0.007285,0.031836,0.027944,0.016068
4,E36007059,Eastbrook,0.211934,0.788066,0.739627,0.007233,0.110297,0.008946,0.018272,0.016178,0.031119,0.007708,0.001237,0.002950,0.021507,0.027122,0.007804


In [14]:
merged2 = pd.merge(merged1, df_ethnicity, how='left', left_on='weird_ward_code', right_on='Code')
merged2 = merged2.drop(['Code', 'Name'], axis=1)
merged2.head()

,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St,weird_ward_code,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,...,Pakistani,Bangladeshi,Indian,Caribbean,Chinese,Latin,Polish,Other,Other_Asian,Other_Black
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South,E36007434,0.690188,0.174941,0.134871,0.206802,...,0.006842,0.001857,0.018473,0.012413,0.006060,0.001662,0.006940,0.020917,0.045939,0.002150
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise,E36007443,0.703854,0.167192,0.128954,0.204617,...,0.024713,0.008136,0.060307,0.015255,0.016068,0.004475,0.009966,0.039459,0.085325,0.000915
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands,E36007430,0.699375,0.151425,0.149200,0.169228,...,0.008799,0.004134,0.034772,0.015160,0.016538,0.002544,0.011449,0.027245,0.054278,0.001166
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra,E36007429,0.657906,0.193705,0.148388,0.208631,...,0.025644,0.007141,0.061242,0.013525,0.016771,0.002597,0.009197,0.050314,0.099762,0.002056
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley,E36007431,0.701454,0.169453,0.129093,0.206648,...,0.022163,0.005442,0.032552,0.013852,0.024735,0.002078,0.019096,0.045216,0.152963,0.003067


In [28]:
df_gender[df_gender.Code=='E36007436']

,Code,Name,Male,Female
381,E36007436,Coombe Vale,0.48,0.52


In [16]:
df_gender=pd.read_excel('Gender_percentage.xlsx', sheet_name = "df")
df_gender.head()

,Code,Name,Male,Female
0,E36007055,Abbey,0.533552,0.466448
1,E36007056,Alibon,0.476745,0.523255
2,E36007057,Becontree,0.486444,0.513556
3,E36007058,Chadwell Heath,0.474603,0.525397
4,E36007059,Eastbrook,0.482296,0.517704


In [17]:
merged3 = pd.merge(merged2, df_gender, how='left', left_on='weird_ward_code', right_on='Code')
merged3 = merged3.drop(['Code', 'Name'], axis=1)
merged3.head()

,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St,weird_ward_code,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,...,Indian,Caribbean,Chinese,Latin,Polish,Other,Other_Asian,Other_Black,Male,Female
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South,E36007434,0.690188,0.174941,0.134871,0.206802,...,0.018473,0.012413,0.006060,0.001662,0.006940,0.020917,0.045939,0.002150,0.485731,0.514269
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise,E36007443,0.703854,0.167192,0.128954,0.204617,...,0.060307,0.015255,0.016068,0.004475,0.009966,0.039459,0.085325,0.000915,0.486220,0.513780
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands,E36007430,0.699375,0.151425,0.149200,0.169228,...,0.034772,0.015160,0.016538,0.002544,0.011449,0.027245,0.054278,0.001166,0.499735,0.500265
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra,E36007429,0.657906,0.193705,0.148388,0.208631,...,0.061242,0.013525,0.016771,0.002597,0.009197,0.050314,0.099762,0.002056,0.497188,0.502812
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley,E36007431,0.701454,0.169453,0.129093,0.206648,...,0.032552,0.013852,0.024735,0.002078,0.019096,0.045216,0.152963,0.003067,0.479474,0.520526


### Smoking

In [18]:
df_smoking=pd.read_csv('df_smoking.csv')
df_smoking=df_smoking[['code', 'currentSmoker', 'exSmoker', 'currentSmokerP10', 'exSmokerP10', 'currentSmokerP100', 'exSmokerP100']]
print(df_smoking.shape)
df_smoking.head()

(45, 7)


,code,currentSmoker,exSmoker,currentSmokerP10,exSmokerP10,currentSmokerP100,exSmokerP100
0,E09000002,0.185714,0.200000,10,3,91,30
1,E09000003,0.171329,0.174825,9,2,89,14
2,E09000004,0.117647,0.256684,2,8,12,73
3,E09000005,0.155844,0.151515,7,1,62,7
4,E09000006,0.137097,0.250000,4,7,37,69


In [27]:
df_smoking[df_smoking.code=='E09000021']

,code,currentSmoker,exSmoker,currentSmokerP10,exSmokerP10,currentSmokerP100,exSmokerP100
19,E09000021,0.153846,0.184615,6,2,57,19


In [19]:
merged4=pd.merge(merged3, df_smoking, how='left', left_on='LB_GSS_CD', right_on='code')
merged4=merged4.drop('code', axis=1)
merged4.head()

,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St,weird_ward_code,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,...,Other_Asian,Other_Black,Male,Female,currentSmoker,exSmoker,currentSmokerP10,exSmokerP10,currentSmokerP100,exSmokerP100
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South,E36007434,0.690188,0.174941,0.134871,0.206802,...,0.045939,0.002150,0.485731,0.514269,0.153846,0.184615,6.0,2.0,57.0,19.0
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise,E36007443,0.703854,0.167192,0.128954,0.204617,...,0.085325,0.000915,0.486220,0.513780,0.153846,0.184615,6.0,2.0,57.0,19.0
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands,E36007430,0.699375,0.151425,0.149200,0.169228,...,0.054278,0.001166,0.499735,0.500265,0.153846,0.184615,6.0,2.0,57.0,19.0
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra,E36007429,0.657906,0.193705,0.148388,0.208631,...,0.099762,0.002056,0.497188,0.502812,0.153846,0.184615,6.0,2.0,57.0,19.0
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley,E36007431,0.701454,0.169453,0.129093,0.206648,...,0.152963,0.003067,0.479474,0.520526,0.153846,0.184615,6.0,2.0,57.0,19.0


### BMI/Obesity

In [20]:
df_bmi=pd.read_excel('Obesity_percentage.xlsx', sheet_name = "df")
df_bmi=df_bmi[['Ward_Code', 'NormalWeight', 'ExcessWeight(Obese&Overweight)']]
df_bmi.head()

,Ward_Code,NormalWeight,ExcessWeight(Obese&Overweight)
0,E05009288,0.381046,0.618954
1,E05009297,0.381046,0.618954
2,E05009302,0.381046,0.618954
3,E05009308,0.381046,0.618954
4,E05000026,0.365039,0.634961


In [29]:
df_bmi[df_bmi.Ward_Code=='E05000579']

,Ward_Code,NormalWeight,ExcessWeight(Obese&Overweight)


In [21]:
merged5 = pd.merge(merged4, df_bmi, how='left', left_on='GSS_CODE', right_on='Ward_Code')
merged5 = merged5.drop('Ward_Code', axis=1)
merged5.head()

,NAME,GSS_CODE,LB_GSS_CD,BOROUGH,NAME_St,weird_ward_code,age_Light_Risk,age_Medium_Risk,age_High_Risk,0_15,...,Male,Female,currentSmoker,exSmoker,currentSmokerP10,exSmokerP10,currentSmokerP100,exSmokerP100,NormalWeight,ExcessWeight(Obese&Overweight)
0,Chessington South,E05000405,E09000021,Kingston upon Thames,Chessington South,E36007434,0.690188,0.174941,0.134871,0.206802,...,0.485731,0.514269,0.153846,0.184615,6.0,2.0,57.0,19.0,0.448914,0.551086
1,Tolworth and Hook Rise,E05000414,E09000021,Kingston upon Thames,Tolworth and Hook Rise,E36007443,0.703854,0.167192,0.128954,0.204617,...,0.486220,0.513780,0.153846,0.184615,6.0,2.0,57.0,19.0,0.448914,0.551086
2,Berrylands,E05000401,E09000021,Kingston upon Thames,Berrylands,E36007430,0.699375,0.151425,0.149200,0.169228,...,0.499735,0.500265,0.153846,0.184615,6.0,2.0,57.0,19.0,0.448914,0.551086
3,Alexandra,E05000400,E09000021,Kingston upon Thames,Alexandra,E36007429,0.657906,0.193705,0.148388,0.208631,...,0.497188,0.502812,0.153846,0.184615,6.0,2.0,57.0,19.0,0.448914,0.551086
4,Beverley,E05000402,E09000021,Kingston upon Thames,Beverley,E36007431,0.701454,0.169453,0.129093,0.206648,...,0.479474,0.520526,0.153846,0.184615,6.0,2.0,57.0,19.0,0.448914,0.551086


In [38]:
df_final.columns

Index(['NAME', 'GSS_CODE', 'LB_GSS_CD', 'BOROUGH', 'NAME_St',
       'weird_ward_code', 'age_Light_Risk', 'age_Medium_Risk', 'age_High_Risk',
       '0_15', '16_24', '25_49', '50_64', '65_74', '75_over', 'Risky',
       'No_Risk', 'White', 'White African', 'Black African', 'Pakistani',
       'Bangladeshi', 'Indian', 'Caribbean', 'Chinese', 'Latin', 'Polish',
       'Other', 'Other_Asian', 'Other_Black', 'Male', 'Female',
       'currentSmoker', 'exSmoker', 'currentSmokerP10', 'exSmokerP10',
       'currentSmokerP100', 'exSmokerP100', 'NormalWeight',
       'ExcessWeight(Obese&Overweight)'],
      dtype='object')